In [10]:
import tensorly as tl
from tensorly.decomposition import *
import numpy as np

In [3]:
tensor = tl.tensor(np.arange(24).reshape((3, 4, 2)), dtype=tl.float64)

In [5]:
noise = tl.tensor(np.random.normal(loc=0.0, scale=1.0, size=(3,4,2)))

In [7]:
tensor, noise

(array([[[ 0.,  1.],
         [ 2.,  3.],
         [ 4.,  5.],
         [ 6.,  7.]],
 
        [[ 8.,  9.],
         [10., 11.],
         [12., 13.],
         [14., 15.]],
 
        [[16., 17.],
         [18., 19.],
         [20., 21.],
         [22., 23.]]]),
 array([[[ 2.78752979e-02, -7.93600200e-01],
         [-1.90287840e+00, -5.32584200e-01],
         [-7.00455731e-01, -1.69541825e-01],
         [ 1.93943086e+00,  1.27977697e-04]],
 
        [[-3.64059633e-02, -3.84900476e-01],
         [ 2.18249318e-01, -4.58355652e-01],
         [-4.73355648e-02, -6.89360537e-01],
         [ 9.77605638e-02,  6.82000123e-01]],
 
        [[ 1.83419886e+00,  4.58955907e-01],
         [ 8.65315927e-01,  4.05372191e-01],
         [-1.23678324e+00, -2.24022834e-01],
         [-4.60143454e-01, -1.20177021e+00]]]))

In [8]:
nz = tensor + noise

In [9]:
nz

array([[[ 0.0278753 ,  0.2063998 ],
        [ 0.0971216 ,  2.4674158 ],
        [ 3.29954427,  4.83045818],
        [ 7.93943086,  7.00012798]],

       [[ 7.96359404,  8.61509952],
        [10.21824932, 10.54164435],
        [11.95266444, 12.31063946],
        [14.09776056, 15.68200012]],

       [[17.83419886, 17.45895591],
        [18.86531593, 19.40537219],
        [18.76321676, 20.77597717],
        [21.53985655, 21.79822979]]])

In [12]:
x = CP_NN_HALS(4).fit_transform(nz)

In [17]:
x[1][0].shape, x[1][1].shape, x[1][2].shape

((3, 4), (4, 4), (2, 4))

In [18]:
from tensorly import tenalg

In [26]:
R = 4
T = tl.tensor(np.zeros((3,4,2), dtype=tl.float64))
for i in range(R):
    T += tenalg.outer((x[1][0][:,i], x[1][1][:,i], x[1][2][:,i]))

In [33]:
tl.norm((T - tensor), 2)

4.063976330341656

In [57]:
def meth(algo, noise, X, rank=4):
    if not issubclass(algo,CP_NN):
        x = algo(rank).fit_transform(noise)
    else:
        x = algo(rank).fit_transform(rank)
    T = tl.tensor(np.zeros((3,4,2), dtype=tl.float64))
    for i in range(rank):
        T += tenalg.outer((x[1][0][:,i], x[1][1][:,i], x[1][2][:,i]))
    return tl.norm((T - X), 2)

In [49]:
meth(CP_NN_HALS, nz, tensor)

3.978575846479921

In [50]:
meth(CP, nz, tensor)

4.209555107852045

In [51]:
meth(CPPower, nz, tensor)

64.7170320454428

In [1]:
from sklearn.datasets import fetch_20newsgroups

In [12]:
newsgroups_train = fetch_20newsgroups(subset='train', categories=["misc.forsale", "alt.atheism", "sci.space", "rec.sport.baseball"])

In [13]:
from pprint import pprint

In [14]:
pprint(list(newsgroups_train.target_names))

['alt.atheism', 'misc.forsale', 'rec.sport.baseball', 'sci.space']


In [29]:
docs = {0:[], 1: [], 2: [], 3: []}
for i in range(len(newsgroups_train.target)):
    docs[newsgroups_train.target[i]].append(newsgroups_train.data[i]) 

In [33]:
len(docs[0]), len(docs[1]), len(docs[2]), len(docs[3])

(480, 585, 597, 593)